# 14. Movielens 영화 추천 실습

## 데이터 준비 및 전처리

2000년 당시 MovieLens 가입자 6,040명의 약 3,900편의 영화에 대한 1,000,209개의 평가를 포함하고 있는 [MovieLens 1M Dataset](https://grouplens.org/datasets/movielens/1m/)을 사용합니다.

In [1]:
import pandas as pd
import os

rating_file_path = os.getenv("HOME") + "/aiffel/recommendata_iu/data/ml-1m/ratings.dat"
ratings_cols = ["user_id", "movie_id", "ratings", "timestamp"]
ratings = pd.read_csv(
    rating_file_path,
    sep="::",
    names=ratings_cols,
    engine="python",
    encoding="ISO-8859-1",
)
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


영화 추천에 사용하지 않을 `timestamp` column을 제거합니다.

In [2]:
ratings = ratings.drop("timestamp", axis=1)
ratings.head()

,user_id,movie_id,ratings
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


3점 미만의 영화는 선호하지 않는 영화로 판단, 추천 항목에 포함되지 않는 것이 바람직하여 데이터에서 제거합니다.

In [3]:
ratings = ratings[ratings["ratings"] >= 3]
filtered_data_size = len(ratings)

print(
    f"orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}"
)
print(f"Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}")

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


여기서는 평점을 시청 횟수로 치환하여 많이 시청하였다는 것은 선호도가 높다는 것으로 간주합니다.

In [4]:
ratings.rename(columns={"ratings": "counts"}, inplace=True)
ratings["counts"]
ratings.head()

,user_id,movie_id,counts
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


3점 미만의 영화를 제거한 뒤의 고유한 영화와 고유한 사용자가 얼마나 존재하는지 확인합니다.

In [5]:
print("# of movie_id:", ratings["movie_id"].nunique())
print("# of user_id :", ratings["user_id"].nunique())

# of movie_id: 3628
# of user_id : 6039


현재 사용하고 있는 데이터셋은 영화의 제목을 문자열이 아니라 인덱스로 나타내고 있음으로, 문자열로된 제목 정보를 가지고 있는 데이터를 불러옵니다.

In [6]:
movie_file_path = os.getenv("HOME") + "/aiffel/recommendata_iu/data/ml-1m/movies.dat"
cols = ["movie_id", "title", "genre"]
movies = pd.read_csv(
    movie_file_path, sep="::", names=cols, engine="python", encoding="ISO-8859-1"
)
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


문자열로 된 제목을 가지고 있는 데이터프레임과 시청 횟수가 담겨있는 데이터프레임을 합칩니다.

In [7]:
ratings = ratings.join(movies.set_index("movie_id"), on="movie_id")
ratings.head()

,user_id,movie_id,counts,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,My Fair Lady (1964),Musical|Romance
3,1,3408,4,Erin Brockovich (2000),Drama
4,1,2355,5,"Bug's Life, A (1998)",Animation|Children's|Comedy


문자열로 된 제목 정보를 가지고 있기에 불필요해진 `movie_id`와 추천에 필요하지 않은 정보 `genre`를 제거합니다.

In [8]:
ratings = ratings.drop(columns=["genre", "movie_id"])
ratings.head()

,user_id,counts,title
0,1,5,One Flew Over the Cuckoo's Nest (1975)
1,1,3,James and the Giant Peach (1996)
2,1,3,My Fair Lady (1964)
3,1,4,Erin Brockovich (2000)
4,1,5,"Bug's Life, A (1998)"


전체 이용자에게 가장 인기있는 영화 30개를 확인합니다.

In [9]:
movie_count = ratings.groupby("title")["user_id"].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

사용자별로 영화를 몇 편씩이나 시청하였는지 통계를 확인합니다.

In [10]:
user_count = ratings.groupby("user_id")["title"].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

사용자별로 한 영화를 몇 번씩 시청하였는지 통계를 확인합니다.

In [11]:
# 유저별 play횟수 중앙값에 대한 통계
user_median = ratings.groupby("user_id")["counts"].median()
user_median.describe()

count    6039.000000
mean        4.055970
std         0.432143
min         3.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         5.000000
Name: counts, dtype: float64

데이터에 존재하지 않는 임의의 사용자를 만들어 시청 횟수 데이터를 추가하고자 합니다.  
우선 추가하고자 하는 영화의 존재 여부를 확인합니다.

In [12]:
movies[
    movies["title"]
    .str.lower()
    .str.contains("forrest | gravity | wall | redemption | hotel")
]

,movie_id,title,genre
352,356,Forrest Gump (1994),Comedy|Romance|War
1278,1298,Pink Floyd - The Wall (1982),Drama|Musical|War
1860,1929,Grand Hotel (1932),Drama
2823,2892,New Rose Hotel (1998),Action|Drama
3403,3472,Horror Hotel (a.k.a. The City of the Dead) (1960),Horror
3506,3575,Defying Gravity (1997),Drama


임의의 영화 제목에 포함되어 있는 키워드 5개를 검색하였으나, 일부 영화만 검색이 됩니다.  
검색이 되는 영화는 2000년 이전의 영화들입니다.

In [13]:
years = movies["title"].str.extract(r"(\([0-9]{4}\))", expand=False)
years.sort_values(ascending=False).head()

3882    (2000)
3528    (2000)
3577    (2000)
3170    (2000)
3555    (2000)
Name: title, dtype: object

한국에서는 "쇼생크 탈출"이라는 제목으로 알려져 있는 "Shawshank Redemption"을 검색합니다.

In [14]:
movies[movies["title"].str.lower().str.contains("redemption")]

,movie_id,title,genre
315,318,"Shawshank Redemption, The (1994)",Drama


브래드 피트와 모건 프리먼 주연의 영화 "세븐"을 검색합니다.

In [15]:
movies[movies["title"].str.lower().str.contains("seven")]

,movie_id,title,genre
46,47,Seven (Se7en) (1995),Crime|Thriller
590,594,Snow White and the Seven Dwarfs (1937),Animation|Children's|Musical
1218,1237,"Seventh Seal, The (Sjunde inseglet, Det) (1957)",Drama
1576,1619,Seven Years in Tibet (1997),Drama|War
1825,1894,Six Days Seven Nights (1998),Adventure|Comedy|Romance
1950,2019,Seven Samurai (The Magnificent Seven) (Shichin...,Action|Drama
1994,2063,Seventh Heaven (Le Septième ciel) (1997),Drama|Romance
2145,2214,Number Seventeen (1932),Thriller
2169,2238,Seven Beauties (Pasqualino Settebellezze) (1976),Comedy|Drama
2194,2263,"Seventh Sign, The (1988)",Thriller


로빈 윌리엄스가 키팅 선생님으로 주연을 맡은 "죽은 시인의 사회"를 검색합니다.

In [16]:
movies[movies["title"].str.lower().str.contains("dead poets")]

,movie_id,title,genre
1226,1246,Dead Poets Society (1989),Drama


여러 시리즈가 제작되었던 "인디아나 존스"를 검색합니다.

In [17]:
movies[movies["title"].str.lower().str.contains("indiana")]

,movie_id,title,genre
1271,1291,Indiana Jones and the Last Crusade (1989),Action|Adventure
2046,2115,Indiana Jones and the Temple of Doom (1984),Action|Adventure


검색을 통해 데이터셋에 존재함이 확인된 5편의 영화에 대해 임의의 사용자 "7000"의 시청 횟수 데이터를 추가합니다.

In [18]:
my_ratings = [5, 4, 4, 4, 4]
my_titles = [
    "Forrest Gump (1994)",
    "Shawshank Redemption, The (1994)",
    "Seven (Se7en) (1995)",
    "Dead Poets Society (1989)",
    "Indiana Jones and the Last Crusade (1989)",
]
my_counts = pd.DataFrame(
    {
        "user_id": [7000] * 5,
        "counts": my_ratings,
        "title": my_titles,
    }
)

if not ratings.isin({"user_id": [7000]})["user_id"].any():
    ratings = pd.concat([ratings, my_counts], ignore_index=True)

ratings

,user_id,counts,title
0,1,5,One Flew Over the Cuckoo's Nest (1975)
1,1,3,James and the Giant Peach (1996)
2,1,3,My Fair Lady (1964)
3,1,4,Erin Brockovich (2000)
4,1,5,"Bug's Life, A (1998)"
...,...,...,...
836478,7000,5,Forrest Gump (1994)
836479,7000,4,"Shawshank Redemption, The (1994)"
836480,7000,4,Seven (Se7en) (1995)
836481,7000,4,Dead Poets Society (1989)


`user_id`와 `title`를 정수화하고, 다시 이전 상태로 돌릴 수 있도록 dictionary를 생성합니다.

In [19]:
user_unique = ratings["user_id"].unique()
movie_unique = ratings["title"].unique()

user_to_idx = {v: k for k, v in enumerate(user_unique)}
movie_to_idx = {v: k for k, v in enumerate(movie_unique)}

`user_id`와 `title`를 정수로 변환합니다.

In [20]:
temp_user_data = ratings["user_id"].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):  
    print("user_id column indexing OK!!")
    ratings["user_id"] = temp_user_data  
else:
    print("user_id column indexing Fail!!")

temp_movie_data = ratings["title"].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print("movie_id column indexing OK!!")
    ratings["title"] = temp_movie_data
else:
    print("movie_id column indexing Fail!!")

ratings

user_id column indexing OK!!
movie_id column indexing OK!!


,user_id,counts,title
0,0,5,0
1,0,3,1
2,0,3,2
3,0,4,3
4,0,5,4
...,...,...,...
836478,6039,5,160
836479,6039,4,157
836480,6039,4,220
836481,6039,4,52


모든 사용자가 모든 영화에 대한 시청 횟수를 기록하지 않았을 것이기에 모든 유저에 대한 모든 영화 시청 횟수를 행렬도 만든다면, 많은 0이 존재할 것입니다. CSR maxtrix로 표현하면 이러한 공간 낭비를 줄일 수 있습니다.

In [21]:
from scipy.sparse import csr_matrix

num_user = ratings["user_id"].nunique()
num_movie = ratings["title"].nunique()

csr_data = csr_matrix(
    (ratings.counts, (ratings.user_id, ratings.title)),
    shape=(num_user, num_movie),
)
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 모델 훈련

In [22]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["KMP_DUPLICATE_LIB_OK"] = "True"
os.environ["MKL_NUM_THREADS"] = "1"

AlternatingLeastSquares 모델을 정의합니다. `factors`는 앞서 각 사용자의 평가 개수 통계를 기반으로 설정합니다. 임의로 75%에 해당하는 값인 177에 가장 가까운 178을 `factors`로 설정합니다.

In [23]:
als_model = AlternatingLeastSquares(
    factors=178, regularization=0.01, use_gpu=False, iterations=30, dtype=np.float32
)

In [24]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [25]:
als_model.fit(csr_data_transpose)

  0%|          | 0/30 [00:00<?, ?it/s]

## 모델 성능 평가

모델이 학습을 통해 생성한 벡터들을 확인합니다.

In [26]:
my_fav, forrest = user_to_idx[7000], movie_to_idx["Forrest Gump (1994)"]
my_fav_vec, forrest_vec = (
    als_model.user_factors[my_fav],
    als_model.item_factors[forrest],
)

In [27]:
my_fav_vec

array([ 0.2857044 , -0.8112368 ,  0.04367202, -0.39793256, -0.23452821,
        0.27154276,  0.11978065,  0.03302728, -0.5340198 , -0.2860341 ,
        0.04272611,  0.28992426, -0.23474805, -0.09641947,  0.03644533,
       -0.29689488, -0.03169929,  0.14486495, -0.29955763,  0.05929385,
        0.01495621, -0.05609677,  0.67474604,  0.19486804, -0.25230172,
       -0.44693518,  0.41012576, -0.39580616,  0.33000332,  0.02629472,
        0.09253662,  0.4707257 ,  0.14148118, -0.39073348, -0.01147531,
        0.14624582,  0.76657724,  0.42620102, -0.4111344 , -0.36758772,
        0.09502622, -0.08665612,  0.65427893,  0.12406337, -0.28151998,
       -0.27590784,  0.5742363 ,  0.49495164, -0.06668904,  0.42417654,
        0.09881365, -0.37388486, -0.08901964,  0.09766892, -0.07989257,
       -0.5167717 ,  0.73983383, -0.26619607, -0.17933248, -0.16898088,
       -0.3351637 ,  0.0264889 , -0.51093125, -0.0093144 , -0.3829668 ,
        0.3980091 ,  0.20425646, -0.5807848 ,  0.16118461,  0.17

In [28]:
forrest_vec

array([ 4.81151678e-02, -1.46545200e-02,  1.92889888e-02,  1.87233854e-02,
       -2.10697250e-03,  6.77311048e-02, -1.23708444e-02, -1.89646985e-02,
       -3.53137478e-02,  1.06103336e-02, -3.44851725e-02,  3.79708670e-02,
        2.16052569e-02,  2.16263551e-02,  4.64170575e-02,  5.93066914e-03,
        8.35522171e-03,  2.04920885e-03,  3.63172740e-02, -1.56905875e-02,
        8.06742627e-03,  9.04165488e-03,  5.18760569e-02,  2.79168077e-02,
       -6.05295179e-03, -6.23246096e-03,  2.20543183e-02,  8.74513388e-03,
        4.55802791e-02, -1.80256944e-02,  1.14664137e-02, -7.08200876e-03,
       -4.18572128e-03, -3.13414559e-02,  9.87235643e-03,  2.98776235e-02,
        5.28183095e-02,  1.82690099e-02, -7.80579308e-03, -1.92115568e-02,
        1.12631638e-02, -7.09716929e-03,  2.33121812e-02, -1.27100619e-02,
        1.89371256e-03,  1.73411444e-02,  3.99328843e-02,  7.46787712e-03,
       -4.90067117e-02,  9.65681579e-03,  2.33460381e-03, -2.14012638e-02,
       -5.54641597e-02, -

임의로 추가한 사용자 "7000"에 대한 영화 "Forrest Gump (1994)"의 내적을 확인합니다.

In [29]:
np.dot(my_fav_vec, forrest_vec)

0.7835302

### 선호하는 영화와 그 이외의 영화에 대한 선호도 평가

임의로 추가한 사용자 "7000"에 대해 5가지 영화의 선호도를 확인합니다.

In [30]:
def get_preference(title: str, user=7000):
    my_fav, my_movie = user_to_idx[user], movie_to_idx[title]
    my_fav_vec, my_movie_vec = (
        als_model.user_factors[my_fav],
        als_model.item_factors[my_movie],
    )
    return np.dot(my_fav_vec, my_movie_vec)

In [31]:
my_titles = [
    "Forrest Gump (1994)",
    "Shawshank Redemption, The (1994)",
    "Seven (Se7en) (1995)",
    "Dead Poets Society (1989)",
    "Indiana Jones and the Last Crusade (1989)",
]
for t in my_titles:
    print(t, get_preference(t))

Forrest Gump (1994) 0.7835302
Shawshank Redemption, The (1994) 0.5308162
Seven (Se7en) (1995) 0.45649365
Dead Poets Society (1989) 0.3698184
Indiana Jones and the Last Crusade (1989) 0.49090663


앞서 확인한 5가지의 영화는 사용자 "7000"에 대해 데이터셋에 정보가 존재하였습니다. 이번에는 사용자 "7000"이 시청 횟수를 기록하지 않은 다른 영화에 대한 선호도를 확인합니다. 한국에서는 "뻐꾸기 둥지 위로 날아간 새"라는 제목으로 알려져 있는 "One Flew Over the Cuckoo's Nest"에 대한 선호도를 확인합니다.

In [32]:
get_preference("One Flew Over the Cuckoo's Nest (1975)")

0.031651583

시청 기록에 따르면 해당 영화를 선호할 확률은 매우 낮습니다.

### 비슷한 영화 추천

이번에는 사용자 기반이 아니라 영화를 기반으로 비슷한 영화를 추천하는 성능을 확인합니다. 주어진 영화에 대해 15편의 유사한 영화를 출력합니다.

In [33]:
favorite_movie = "Forrest Gump (1994)"
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)

idx_to_movie = {v: k for k, v in movie_to_idx.items()}
[(idx_to_movie[i[0]], i[1]) for i in similar_movie]

[('Forrest Gump (1994)', 0.99999994),
 ('Pretty Woman (1990)', 0.33306262),
 ('Groundhog Day (1993)', 0.32543892),
 ('Ghost (1990)', 0.30011),
 ('Soft Fruit (1999)', 0.28836837),
 ('As Good As It Gets (1997)', 0.2882182),
 ('Notting Hill (1999)', 0.2820463),
 ('Sleepless in Seattle (1993)', 0.27755955),
 ('Doctor Zhivago (1965)', 0.27345443),
 ('Small Wonders (1996)', 0.27217126),
 ("Devil's Brigade, The (1968)", 0.2713098),
 ('Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)', 0.27102003),
 ('Closer You Get, The (2000)', 0.25924814),
 ('Wedding Singer, The (1998)', 0.2591012),
 ('Gate II: Trespassers, The (1990)', 0.25794888)]

영화 "포레스트 검프"에 대해 15편의 영화를 찾았습니다. 로맨스 영화 "귀여운 여인", "노팅힐" 등이 결과에 포함되어 있습니다.

In [34]:
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [35]:
get_similar_movie("Toy Story (1995)")

['Toy Story (1995)',
 'Toy Story 2 (1999)',
 'Aladdin (1992)',
 "Bug's Life, A (1998)",
 'Babe (1995)',
 'Soft Toilet Seats (1999)',
 'Nobody Loves Me (Keiner liebt mich) (1994)',
 'Lion King, The (1994)',
 'Groundhog Day (1993)',
 'Ballad of Narayama, The (Narayama Bushiko) (1958)']

"토이 스토리"에 대해서는 후속작 "토이 스토리 2"와 "알라딘", "벅스 라이프"와 같은 애니메이션들이 포함되어 있습니다.

### 좋아할 만한 영화 추천

이번에는 사용자를 기반으로 좋아할 만한 영화를 예측해보겠습니다.

In [36]:
user = user_to_idx[7000]
movie_recommended = als_model.recommend(
    user, csr_data, N=20, filter_already_liked_items=True
)
result = [(idx_to_movie[i[0]], i[1]) for i in movie_recommended]
result

[('Raiders of the Lost Ark (1981)', 0.4754969),
 ('Pulp Fiction (1994)', 0.32331765),
 ('Silence of the Lambs, The (1991)', 0.30120814),
 ('Die Hard (1988)', 0.29427224),
 ('Field of Dreams (1989)', 0.28319046),
 ('Indiana Jones and the Temple of Doom (1984)', 0.26308197),
 ('Batman (1989)', 0.26167807),
 ('Good Will Hunting (1997)', 0.25784227),
 ('Usual Suspects, The (1995)', 0.25419387),
 ('Rain Man (1988)', 0.25106233),
 ("Schindler's List (1993)", 0.2485169),
 ('Jerry Maguire (1996)', 0.24257226),
 ('Groundhog Day (1993)', 0.24198993),
 ('Reservoir Dogs (1992)', 0.24036963),
 ('Few Good Men, A (1992)', 0.22032937),
 ('Sixth Sense, The (1999)', 0.19659573),
 ('Breakfast Club, The (1985)', 0.18284252),
 ('Untouchables, The (1987)', 0.179327),
 ('Fargo (1996)', 0.16780315),
 ('Dances with Wolves (1990)', 0.16710861)]

상위 5개의 항목을 살펴보면 한국에는 "레이더스"라고 알려져있는 인디아나 존스의 첫번째 영화가 포함되어 있고, "양들의 침묵", "다이 하드"가 포함되어 있습니다. 액션, 스릴러 영화를 우선적으로 고려한 것으로 보입니다.

위와 같은 예측 결과에는 기존의 어떤 영화들이 영향을 미쳤는지 확인합니다.

In [37]:
recomm_top = movie_to_idx[result[0][0]]
explain = als_model.explain(user, csr_data, itemid=recomm_top)

In [38]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Indiana Jones and the Last Crusade (1989)', 0.39170656465277565),
 ('Shawshank Redemption, The (1994)', 0.0646803601234153),
 ('Dead Poets Society (1989)', 0.03666294103986788),
 ('Seven (Se7en) (1995)', -0.010386620265246442),
 ('Forrest Gump (1994)', -0.012051698270388964)]

추천 결과에 가장 많은 영향을 미친 영화는 "인디아나 존스: 최후의 성전"입니다. 영화 추천 상위에 액션 영화가 포함되어 있는 것이 납득됩니다.

## 추가 실험

앞서 임의로 설정한 `factors`에 대한 추가 실험을 진행합니다. 이번에는 75%에 1.5 * IQR을 더한 384를 사용하여 훈련을 진행하고 추천 결과를 확인합니다.

In [39]:
als_model = AlternatingLeastSquares(
    factors=384, regularization=0.01, use_gpu=False, iterations=30, dtype=np.float32
)
als_model.fit(csr_data_transpose)

  0%|          | 0/30 [00:00<?, ?it/s]

### 영화 선호도 확인

In [40]:
def get_preference(title: str, user=7000):
    my_fav, my_movie = user_to_idx[user], movie_to_idx[title]
    my_fav_vec, my_movie_vec = (
        als_model.user_factors[my_fav],
        als_model.item_factors[my_movie],
    )
    return np.dot(my_fav_vec, my_movie_vec)


my_titles = [
    "Forrest Gump (1994)",
    "Shawshank Redemption, The (1994)",
    "Seven (Se7en) (1995)",
    "Dead Poets Society (1989)",
    "Indiana Jones and the Last Crusade (1989)",
]


for t in my_titles:
    print(t, get_preference(t))

Forrest Gump (1994) 0.9065602
Shawshank Redemption, The (1994) 0.8480511
Seven (Se7en) (1995) 0.70374405
Dead Poets Society (1989) 0.58585453
Indiana Jones and the Last Crusade (1989) 0.70563745


`factors`를 178로 설정한 모델과 비교하여 각 영화에 대한 선호도(내적값)이 커졌습니다. 적어도 해당 유저에 대해서는 선호도에 대한 정확도가 상승한 것으로 보입니다.

### 비슷한 영화 추천

In [41]:
favorite_movie = "Forrest Gump (1994)"
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)

idx_to_movie = {v: k for k, v in movie_to_idx.items()}
[(idx_to_movie[i[0]], i[1]) for i in similar_movie]

[('Forrest Gump (1994)', 1.0),
 ('Soft Fruit (1999)', 0.27223584),
 ("Child's Play 3 (1992)", 0.26744452),
 ("Devil's Brigade, The (1968)", 0.26531988),
 ('Digimon: The Movie (2000)', 0.2581774),
 ('Small Wonders (1996)', 0.25140128),
 ('Death in the Garden (Mort en ce jardin, La) (1956)', 0.2513942),
 ('Nil By Mouth (1997)', 0.25062174),
 ('Outside Ozona (1998)', 0.25039825),
 ('Great Locomotive Chase, The (1956)', 0.25013283),
 ('Ulysses (Ulisse) (1954)', 0.24875452),
 ('King in New York, A (1957)', 0.24771856),
 ('Three Ages, The (1923)', 0.24757968),
 ('Held Up (2000)', 0.24711852),
 ('Anna (1996)', 0.24696955)]

비슷한 영화 추천에서는 "Child's Play 3 (1992)(사탄의 인형 3)", "디지몬"와 같이 다소 납득하기 어려운 결과를 얻었습니다.

### 좋아할 만한 영화 추천

In [42]:
user = user_to_idx[7000]
movie_recommended = als_model.recommend(
    user, csr_data, N=20, filter_already_liked_items=True
)
result = [(idx_to_movie[i[0]], i[1]) for i in movie_recommended]
result

[('Indiana Jones and the Temple of Doom (1984)', 0.24853942),
 ('Raiders of the Lost Ark (1981)', 0.24483553),
 ('Rain Man (1988)', 0.19910486),
 ('Field of Dreams (1989)', 0.1929937),
 ('Die Hard (1988)', 0.19208476),
 ('Batman (1989)', 0.18481672),
 ('Usual Suspects, The (1995)', 0.18255043),
 ('Game, The (1997)', 0.168211),
 ('Amadeus (1984)', 0.155754),
 ('Good Will Hunting (1997)', 0.14517374),
 ('E.T. the Extra-Terrestrial (1982)', 0.14316434),
 ('Outsiders, The (1983)', 0.13902116),
 ('Clerks (1994)', 0.13572203),
 ('Jerry Maguire (1996)', 0.13517123),
 ('Reservoir Dogs (1992)', 0.13154201),
 ('Groundhog Day (1993)', 0.12964217),
 ('Last of the Mohicans, The (1992)', 0.12484228),
 ('Driving Miss Daisy (1989)', 0.1237856),
 ('Dead Calm (1989)', 0.120958805),
 ('On Golden Pond (1981)', 0.117590465)]

좋아할 만한 영화 추천에서는 이전 모델과 비슷하게 "레이더스"와 "다이 하드"를 결과로 얻었습니다.

## 결론

- 임의로 생성한 사용자 데이터에는 5개의 기록만이 존재하지만, 영화 추천 결과를 납득할 수 있었습니다.
- 약 84만건의 기록에 대한 AlternatingLeastSquares 모델의 훈련 시간은 충분히 합리적이었으며, 단시간의 학습을 통해서도 좋은 예측을 할 수 있었습니다.
- AlternatingLeastSquares 모델의 하이퍼파라미터인 `factors`를 변경하는 실험을 진행한 결과 다음과 같은 고찰을 얻을 수 있었습니다.
    - `factors`의 수를 늘리면 특정 유저에 대한 선호도를 정확하게 예측할 확률이 높아집니다. 이는 데이터에 대한 과적합이 발생할 가능성이 높아진다는 이야기와 연결됩니다. 따라서 적절한 하이퍼파라미터를 설정하는 것이 중요합니다.
    - 이번 실험에서는 `factors`를 384로 설정한 것보다 178로 설정한 모델의 예측이 더 합리적이라고 생각됩니다.
- 여러 사용자가 남긴 기록만을 사용하여도 활용 가치가 높은 정보를 얻을 수 있는 것을 보아, 각각의 영화가 갖고 있는 다른 특징들(장르, 개봉일자)을 포함하면 더 정교한 추천이 가능할 것으로 생각됩니다.

## 루브릭

| 평가문항|상세기준|
|---|---|
| 1. CSR matrix가 정상적으로 만들어졌다. | 사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.|
| 2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다. | 사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.|
| 3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다. | MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.|

### 자체 루브릭

1. CSR matrix를 사용자와 아이템 개수를 설정하여 생성하였다.
2. 사용자와 영화에 대한 내적 수치가 의미있는 결과를 보여주었다.
3. 유사 영화 추천에 대해 의미있는 결과를 보여주었다.